In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
import zipfile
import os

base_path = r'C:\Users\91702\Desktop\Food-Waste-Recycling-Management-using-IoT-and-AI-main'
zip_path = os.path.join(base_path, 'archive.zip')
extract_folder = os.path.join(base_path, 'Waste Images')

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f'Files extracted to {extract_folder}')

Files extracted to C:\Users\91702\Desktop\Food-Waste-Recycling-Management-using-IoT-and-AI-main\Waste Images


In [3]:
# List folders in the dataset
dataset_path = extract_folder
for category in os.listdir(dataset_path):
    print(category)

Aluminium
Carton
E-waste
Glass
Organic_Waste
Paper_and_Cardboard
Plastics
Textiles
Waste Images
Wood


***SVM MODEL***

In [ ]:
# Define the categories (folder names)
categories = ['Aluminium', 'Carton', 'E-Waste', 'Glass', 'Organic_Waste', 'Paper_and_Cardboard', 'Plastics', 'Textiles', 'Wood']
image_data = []
labels = []

# Load VGG16 model (without top layers)
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Initialize lists
image_data = []
labels = []

# Load and process images
for category in categories:
    folder_path = os.path.join(dataset_path, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
    
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        try:
            # Resize to 224x224 for VGG16
            img_resized = cv2.resize(img, (224, 224))
            img_array = image.img_to_array(img_resized)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
    
            # Extract features using VGG16
            features = vgg_model.predict(img_array)
            flattened = features.flatten()
    
            image_data.append(flattened)
            labels.append(category)
    
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# Convert lists to numpy arrays
X = np.array(image_data)
y = np.array(labels)

# Encode labels to numeric values (Aluminium -> 0, Carton -> 1, etc.)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM classifier
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"VGG16 + SVM Accuracy: {accuracy * 100:.2f}%")
print("Predicted labels (decoded):", label_encoder.inverse_transform(y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [15]:
# Check if the folder exists before trying to read images
for category in categories:
    folder_path = os.path.join(dataset_path, category)
    
    if os.path.exists(folder_path):
        print(f'Found folder: {category}')
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            
            # Process the image here...
    else:
        print(f'Folder not found: {category}')

Found folder: Aluminium
Found folder: Wood


***Predicting Outcome***

***Extracting CNN Features For DATASET & Training SVM***

***Classifying Image***

In [17]:
def classify_new_image(image_path):
    features = extract_vgg16_features(image_path)  # Extract CNN features
    prediction = svm_model.predict([features])  # Predict using trained SVM
    predicted_label = label_encoder.inverse_transform(prediction)  # Convert back to category
    return predicted_label[0]

# Example: Classify a new image
new_image_path = os.path.join(base_path, r'Project_pictures\meat.jpg')
predicted_category = classify_new_image(new_image_path)
print(f'The new image belongs to the category: {predicted_category}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
The new image belongs to the category: Wood
